# Fix Alteration Filing Metadata

### Environment Set up

In [1]:
%run /workspaces/lear/tests/data/default-bcr-business-setup-TEST.ipynb

/opt/conda/lib/python3.9/site-packages/jose/backends/cryptography_backend.py:18: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes, int_to_bytes
Unable to configure logging, attempted conf:/opt/conda/lib/python3.9/site-packages/legal_api/logging.conf


In [2]:
import dpath
import json
from contextlib import suppress
from legal_api import db 
from legal_api.models import Filing
from sqlalchemy import desc
from sqlalchemy.orm.attributes import flag_modified

### Get Completed Filings by type

In [12]:
def get_completed_filings_by_type(filing_type: str):
    """Return the filings of a particular type."""
    filings = db.session.query(Filing). \
        filter(Filing._filing_type == filing_type). \
        filter(Filing._status == Filing.Status.COMPLETED). \
        order_by(desc(Filing.filing_date)). \
        all()
    return filings

### Update Alteration Filing Metadata

In [10]:
def update_alteration_metadata():     
    completed_alteration_filings = get_completed_filings_by_type('alteration')
    for filing in completed_alteration_filings:
        try:  
            filing_meta_data = filing._meta_data
            filing_meta_data['legalFilings'] = ['alteration']
            print(filing_meta_data)
            # Update the from and to legal type always.
            alteration_meta = {}
            from_legal_type = dpath.util.get(filing._filing_json, '/filing/business/legalType')
            to_legal_type = dpath.util.get(filing._filing_json, '/filing/alteration/business/legalType')        
            if from_legal_type and to_legal_type:           
                alteration_meta = {**alteration_meta, **{'fromLegalType': from_legal_type,
                                                    'toLegalType': to_legal_type}}


            # Update the fromLegalName and toLegalName if there is a name change    
            with suppress(IndexError, KeyError, TypeError):    
                from_legal_name = dpath.util.get(filing._filing_json, '/filing/business/legalName')
                identifier = dpath.util.get(filing._filing_json, '/filing/business/identifier')
                name_request_json = dpath.util.get(filing._filing_json, '/filing/alteration/nameRequest')
                to_legal_name = name_request_json.get('legalName', identifier[2:] + ' B.C. LTD.')             
                if from_legal_name != to_legal_name:
                    alteration_meta = {**alteration_meta, **{'fromLegalName': from_legal_name,
                                                        'toLegalName': to_legal_name}}

            filing_meta_data['alteration'] = alteration_meta            
            filing._meta_data = filing_meta_data
            # For ORM to help identify that the json has changed. 
            flag_modified(filing, "_meta_data")
            
            db.session.add(filing)
            db.session.commit()
            print( f'\033[92m Updated {filing.id} - {filing._meta_data}')
        except Exception as e:             
            print(f'\033[91mUpdate failed for filing Id - {filing.id}')


In [13]:
update_alteration_metadata()

{'alteration': {}, 'legalFilings': ['alteration'], 'applicationDate': '2021-03-23T18:08:30.508214+00:00'}
 Updated 111822 - {'alteration': {'toLegalName': 'CAMERON INSURANCE LTD.', 'toLegalType': 'BC', 'fromLegalName': 'SOFTWARE INSURANCE INC.', 'fromLegalType': 'BC'}, 'legalFilings': ['alteration'], 'applicationDate': '2021-03-23T18:08:30.508214+00:00'}
